# Partie 2 (suite) : Outils et Agents RAG - VéloCorp

Ce notebook présente :
1. **Les 6 Outils** disponibles pour interroger différentes sources
2. **SimpleAgent** : Agent basique qui utilise les outils
3. **ReflectionAgent** : Agent avancé avec critique et amélioration

---

## Setup

In [ ]:
# Installation du package
!pip install git+https://github.com/racousin/rag_attack.git -q

In [ ]:
from rag_attack import (
    set_config,
    SimpleAgent,
    ReflectionAgent,
    get_crm,
    get_erp,
    get_document_rag,
    get_internet_search,
    write_file,
    send_mail,
)

config = {
    # Remplir avec vos credentials
}

set_config(config)

# Liste des outils
all_tools = [get_document_rag, get_erp, get_crm, get_internet_search, write_file, send_mail]
print(f"6 outils disponibles: {[t.name for t in all_tools]}")

---
# Chapitre 1 : Les 6 Outils

| Outil | Source | Usage |
|-------|--------|-------|
| `get_crm` | API REST | Opportunités, prospects, commerciaux |
| `get_erp` | SQL Server | Produits, clients, commandes |
| `get_document_rag` | Azure Search | Documents, FAQs, manuels |
| `get_internet_search` | DuckDuckGo | Recherche web |
| `write_file` | Local | Génération de rapports |
| `send_mail` | SMTP | Envoi d'emails |

## 1.1 CRM Tool (`get_crm`)

Accès aux données commerciales via API REST.

In [ ]:
print(get_crm.name)
print(get_crm.description)
print("\nArgs:", get_crm.args)

In [ ]:
# Exemples d'utilisation
# get_crm.invoke({"query_type": "analytics"})  # KPIs globaux
# get_crm.invoke({"query_type": "opportunities", "status": "Négociation", "min_value": 1000})
# get_crm.invoke({"query_type": "prospects", "min_score": 70})
# get_crm.invoke({"query_type": "sales_reps", "region": "Bretagne"})

## 1.2 ERP Tool (`get_erp`)

Requêtes SQL sur la base Azure SQL Server.

**Tables disponibles:**
- `products`: id, model, name, price, stock_quantity...
- `customers`: id, customer_type, name, email, total_spent...
- `orders`: id, customer_id, status, total_amount...
- `order_items`, `support_tickets`, `invoices`...

In [ ]:
print(get_erp.name)
print(get_erp.description)

In [ ]:
# Exemples d'utilisation (T-SQL syntax)
# get_erp.invoke({"query": "SELECT TOP 5 name, price FROM products ORDER BY price DESC"})
# get_erp.invoke({"query": "SELECT COUNT(*) as total FROM customers"})
# get_erp.invoke({"query": "SELECT status, COUNT(*) as nb FROM orders GROUP BY status"})

## 1.3 Document RAG (`get_document_rag`)

Recherche dans Azure Cognitive Search.

**Types de recherche:**
- `keyword`: Termes exacts, noms de produits
- `vector`: Questions conceptuelles, sémantique
- `hybrid`: Combinaison des deux (par défaut)

In [ ]:
print(get_document_rag.name)
print(get_document_rag.description)

In [ ]:
# Exemples d'utilisation
# get_document_rag.invoke({"query": "E-City", "search_type": "keyword"})
# get_document_rag.invoke({"query": "comment entretenir batterie", "search_type": "vector"})
# get_document_rag.invoke({"query": "problèmes freins", "search_type": "hybrid"})

## 1.4 Internet Search (`get_internet_search`)

Recherche web via DuckDuckGo (pas de clé API requise).

In [ ]:
print(get_internet_search.name)
print(get_internet_search.description)

In [ ]:
# Exemples d'utilisation
# get_internet_search.invoke({"query": "vélo électrique tendances 2024"})
# get_internet_search.invoke({"query": "réglementation vélos France", "num_results": 3})

## 1.5 Writer Tool (`write_file`)

Génération de rapports (texte ou Excel).

In [ ]:
print(write_file.name)
print(write_file.description)

In [ ]:
# Exemples d'utilisation
# write_file.invoke({"title": "Analyse Q4", "content": "Ventes +15%...", "report_type": "business"})
# write_file.invoke({"title": "Stock", "content": "Produit||Prix||Stock\nE-City||1299||45", "format": "excel"})

## 1.6 Mail Tool (`send_mail`)

Envoi d'emails via SMTP.

In [ ]:
print(send_mail.name)
print(send_mail.description)

In [ ]:
# Exemples d'utilisation
# send_mail.invoke({"to": "email@example.com", "subject": "Rapport", "body": "Contenu..."})
# send_mail.invoke({"to": "email@example.com", "subject": "Rapport", "body": "<h1>HTML</h1>", "html": True})

---
# Chapitre 2 : SimpleAgent

L'agent le plus simple : il **décide** quel outil utiliser et génère une réponse.

```
Question → [Choix outil] → [Exécution] → Réponse
```

In [ ]:
agent = SimpleAgent(
    config=config,
    tools=[get_document_rag, get_erp, get_crm, get_internet_search],
    max_iterations=5,
    verbose="normal"  # "silent", "normal", "verbose"
)

# Visualiser le graphe
agent.display_graph()

## 2.1 Test : Questions Documentation

In [ ]:
question = "Quelle est la durée de garantie des vélos VéloCorp?"
print(f"Question: {question}\n")

response = agent.invoke(question)
print(f"\nReponse:\n{response}")

## 2.2 Test : Questions ERP (SQL)

In [ ]:
question = "Quels sont les 3 vélos les plus chers en stock?"
print(f"Question: {question}\n")

response = agent.invoke(question)
print(f"\nReponse:\n{response}")

## 2.3 Test : Questions CRM

In [ ]:
question = "Combien y a-t-il de commerciaux dans l'équipe?"
print(f"Question: {question}\n")

response = agent.invoke(question)
print(f"\nReponse:\n{response}")

## 2.4 Test : Questions Multi-Sources

L'agent peut utiliser **plusieurs outils** pour une question complexe.

In [ ]:
question = "Donne le prix du E-City et ses caractéristiques techniques"
print(f"Question: {question}\n")

response = agent.invoke(question)
print(f"\nReponse:\n{response}")

## 2.5 Génération de Rapports

L'agent peut créer des fichiers avec l'outil `write_file`.

In [ ]:
agent_writer = SimpleAgent(
    config=config,
    tools=[get_erp, get_crm, write_file],
    max_iterations=5,
    verbose="normal"
)

question = "Génère un rapport Excel des 5 produits les plus chers avec leur stock"
print(f"Question: {question}\n")

response = agent_writer.invoke(question)
print(f"\nReponse:\n{response}")

## 2.6 Envoi d'Emails

L'agent peut envoyer des emails avec l'outil `send_mail`.

In [ ]:
agent_mailer = SimpleAgent(
    config=config,
    tools=[get_erp, get_crm, send_mail],
    max_iterations=5,
    verbose="normal"
)

# REMPLACEZ par votre email pour tester
MON_EMAIL = "votre.email@example.com"

# Décommentez pour envoyer
# question = f"Envoie un email à {MON_EMAIL} avec le résumé des 3 vélos les plus chers"
# response = agent_mailer.invoke(question)
# print(response)

---
# Chapitre 3 : ReflectionAgent

Pattern avancé : l'agent génère une réponse, la **critique**, puis l'**améliore**.

```
Question → [Outils] → Réponse initiale → Critique → Réponse améliorée
```

Ce pattern produit des réponses de **meilleure qualité** pour les questions complexes.

In [ ]:
reflection_agent = ReflectionAgent(
    config=config,
    tools=[get_erp, get_crm, get_document_rag],
    max_iterations=5,
    verbose="normal"
)

# Visualiser le graphe (plus complexe que SimpleAgent)
reflection_agent.display_graph()

In [ ]:
question = "Analyse la situation commerciale de VéloCorp: produits phares, stock, et pipeline"
print(f"Question: {question}\n")

response = reflection_agent.invoke(question)
print(f"\nReponse finale:\n{response}")

In [ ]:
# Voir les détails de la réflexion
details = reflection_agent.get_last_run()

print("=== DÉTAILS DE LA RÉFLEXION ===\n")
print("1. Réponse initiale:")
print(details["initial_response"][:400] + "..." if details["initial_response"] else "N/A")
print("\n2. Critique:")
print(details["critique"][:400] + "..." if details["critique"] else "N/A")
print("\n3. Réponse améliorée:")
print(details["improved_response"][:400] + "..." if details["improved_response"] else "N/A")

---
# Chapitre 4 : Personnalisation

## 4.1 Voir les prompts actuels

In [ ]:
print("=== PROMPTS SIMPLEAGENT ===")
for name, prompt in agent.get_prompts().items():
    print(f"\n{name}:")
    print(f"  {prompt[:100]}...")

## 4.2 Agent avec prompt personnalisé

In [ ]:
custom_agent = SimpleAgent(
    config=config,
    tools=[get_document_rag, get_erp, get_crm],
    system_prompt="""Tu es l'assistant expert VéloCorp.
IMPORTANT: Réponds toujours avec des bullet points et cite tes sources.""",
    max_iterations=5,
    verbose="normal"
)

question = "Quels sont les vélos électriques disponibles?"
response = custom_agent.invoke(question)
print(response)

---
# Chapitre 5 : Comparaison SimpleAgent vs ReflectionAgent

In [ ]:
import time

# Agents en mode silencieux
simple = SimpleAgent(config=config, tools=[get_erp, get_crm], max_iterations=5, verbose="silent")
reflect = ReflectionAgent(config=config, tools=[get_erp, get_crm], max_iterations=5, verbose="silent")

question = "Analyse des ventes: top produits et opportunités en cours"
print(f"Question: {question}\n")

# SimpleAgent
start = time.time()
r1 = simple.invoke(question)
t1 = time.time() - start
print(f"=== SIMPLEAGENT ({t1:.1f}s) ===")
print(r1[:400] + "...\n")

# ReflectionAgent
start = time.time()
r2 = reflect.invoke(question)
t2 = time.time() - start
print(f"=== REFLECTIONAGENT ({t2:.1f}s) ===")
print(r2[:400] + "...")

print(f"\nDifférence: +{t2-t1:.1f}s pour la réflexion")

---
## Résumé

### Les 6 Outils
| Outil | Usage |
|-------|-------|
| `get_crm` | Données commerciales (API) |
| `get_erp` | Base de données SQL |
| `get_document_rag` | Recherche documentaire |
| `get_internet_search` | Recherche web |
| `write_file` | Génération de rapports |
| `send_mail` | Envoi d'emails |

### Les 2 Agents
| Agent | Quand l'utiliser |
|-------|------------------|
| `SimpleAgent` | Questions simples, rapidité |
| `ReflectionAgent` | Questions complexes, qualité |

### Prochaines étapes
- Expérimentez avec différentes combinaisons d'outils
- Personnalisez les prompts pour votre cas d'usage
- Testez sur vos propres questions

---
## Banque de Questions à Tester

In [ ]:
questions = {
    "Documentation": [
        "Quelle est la procédure de garantie?",
        "Comment entretenir la batterie d'un vélo électrique?",
    ],
    "ERP/SQL": [
        "Top 5 des produits par prix",
        "Clients avec plus de 5000€ de dépenses",
    ],
    "CRM": [
        "Opportunités > 5000€",
        "Performance des commerciaux",
    ],
    "Multi-sources": [
        "Analyse complète du E-City: prix, specs, opportunités",
        "Produits à risque: stock faible + forte demande",
    ],
}

print("Questions disponibles:")
for cat, qs in questions.items():
    print(f"\n{cat}:")
    for q in qs:
        print(f"  - {q}")